In [1]:
import hydra
import torch
import pytorch_lightning as pl
from hydra.core.global_hydra import GlobalHydra
import itertools

In [2]:
data_dir = 'D:/open/'

config_yaml_path = "logs/runs/efficientnet_b0/2022-04-07/17-22-47/.hydra/config.yaml"
ckpt_path = 'logs/runs/efficientnet_b0/2022-04-07/17-22-47/checkpoints/epoch=00.ckpt'

submission_file_name = 'submission'


In [7]:
def load_model_with_config(config_path, checkpoint_path):
    config_path = config_path.replace('\\','/')
    config_dir = '/'.join(config_path.split('/')[:-1])
    config_name = config_path.split('/')[-1]
    GlobalHydra.instance().clear()
    hydra.initialize(config_path = config_dir)
    cfg = hydra.compose(config_name=config_name)
    model = hydra.utils.instantiate(cfg['framework'])
    model.load_state_dict(torch.load(checkpoint_path, map_location='cpu')['state_dict'])
    return model

def load_datasets_with_config(config_path, data_dir, num_workers=0):
    config_path = config_path.replace('\\','/')
    config_dir = '/'.join(config_path.split('/')[:-1])
    config_name = config_path.split('/')[-1]
    GlobalHydra.instance().clear()
    hydra.initialize(config_path = config_dir)
    cfg = hydra.compose(config_name=config_name)
    datasets = hydra.utils.instantiate(cfg['dataloader']['datasets'],
                                       data_root=data_dir,
                                       is_test=True,
                                       num_workers=num_workers)
    return datasets

def get_pred(model, test_data_loader, gpus=[0]):
    trainer = pl.Trainer(gpus=gpus)
    preds = trainer.predict(model, test_data_loader)
    img_idx = list(itertools.chain(*[list(pred[0]) for pred in preds]))
    preds = torch.cat([pred[1] for pred in preds], dim=0)
    return img_idx, preds

In [8]:
# load_model/dataloader
model = load_model_with_config(config_yaml_path, ckpt_path)
datasets = load_datasets_with_config(config_yaml_path, data_dir)
test_loader =datasets.get_test_dataloaders()

In [9]:
img_idx, pred_scores = get_pred(model, test_loader)
pred_scores = torch.argmax(pred_scores, dim=-1).detach().cpu().numpy()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\user\anaconda3\envs\abnormal2\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, predict dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

IndexError: list index out of range

In [9]:
idx2label = datasets.get_idx2label()
with open(f'{submission_file_name}.csv', 'w') as f:
    f.write('index,label\n')
    for idx, pred_class in zip(img_idx, pred_scores):
        f.write(f'{idx},{idx2label[pred_class]}\n')


(('16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31'),
 tensor([[1.1001e-05, 1.0479e-05, 1.0774e-06,  ..., 2.6837e-05, 6.0762e-06,
          4.2351e-06],
         [2.7773e-05, 1.3224e-04, 1.4810e-04,  ..., 5.9882e-05, 2.3941e-04,
          9.0928e-05],
         [1.2573e-04, 4.5162e-04, 2.6498e-04,  ..., 1.4838e-04, 7.5741e-04,
          2.0776e-03],
         ...,
         [3.2622e-06, 3.3817e-06, 6.4151e-06,  ..., 9.9302e-06, 7.5967e-06,
          9.2353e-06],
         [6.3186e-06, 1.6952e-05, 1.6690e-04,  ..., 1.6599e-05, 2.0488e-06,
          2.9576e-05],
         [1.8927e-04, 2.4346e-04, 1.0748e-04,  ..., 3.1579e-04, 6.7759e-05,
          6.3946e-05]], device='cuda:0'))